In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
import time
from tqdm import tqdm
from IPython.display import Image
import regex as re
from IPython.core.display import display,HTML
from joblib import Parallel, delayed
from datetime import datetime

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
PATH=ChromeDriverManager().install()

In [3]:
# EL REFUGIO

In [4]:
url = 'https://elrefugio.org/adopta/perros'
driver=webdriver.Chrome(PATH) 
driver.get(url)
time.sleep(3)

muchos=driver.find_elements(By.CLASS_NAME, 'item')
uno = [e.find_element(By.TAG_NAME, 'a') for e in muchos]
links_elrefugio = [e.get_attribute('href') for e in uno]

imagen=[]
nombres= []
carac= []
protec_id=[]
for e in tqdm(links_elrefugio):
    protec_id.append(1)
    driver.get(e)
    box = driver.find_element(By.CLASS_NAME, 'boxSlideFotos')
    img = box.find_element(By.TAG_NAME, 'img')
    imagen.append(img.get_attribute('src'))
    desc = driver.find_element(By.CLASS_NAME, 'txtDoble')
    nombres.append(desc.find_element(By.TAG_NAME, 'h1').text)
    todo = desc.find_elements(By.TAG_NAME, 'li')
    for i in todo:
        i = i.text
        i = i.split(':')
        i.pop(0)
        carac += i
        
driver.quit()

n = 6
caracter = [carac[i:i + n] for i in range(0, len(carac), n)]

100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [00:49<00:00,  1.44it/s]


In [5]:
colnames= ['SEXO', 'EDAD', 'NACIMIENTO', 'RAZA', 'TAMAÑO', 'CARACTER']
refugio = pd.DataFrame(caracter, columns=colnames)

refugio['NOMBRE']= nombres
refugio['PROTECTORA']= protec_id
refugio['LINK']= links_elrefugio

sexo= []
for e in refugio.SEXO:
    e = e.lower()
    e= e.strip()
    sexo.append(e)
refugio['SEXO']= sexo

raza= []
for e in refugio.RAZA:
    e = e.lower()
    e= e.strip()
    raza.append(e)
refugio['RAZA']= raza

tam= []
for e in refugio.TAMAÑO:
    e = e.lower()
    e= e.strip()
    tam.append(e)
refugio['TAMAÑO']= tam

car= []
for e in refugio.CARACTER:
    e = e.lower()
    e= e.strip()
    car.append(e)
refugio['CARACTER']= car

nom= []
for e in refugio.NOMBRE:
    e = e.capitalize()
    e= e.strip()
    nom.append(e)
refugio['NOMBRE']= nom

nac= []
for e in refugio.NACIMIENTO:
    e= e.strip()
    nac.append(e)
refugio['NACIMIENTO']= nac




images1 = imagen
refugio['FOTO'] = images1

# convert your links to html tags 
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

pd.set_option('display.max_colwidth', None)

image_cols = ['FOTO']  #<- define which columns will be used to convert to html

# Create the dictionariy to be passed as formatters
format_dict = {}
for e in image_cols:
    format_dict[e] = path_to_image_html


#display(HTML(perros.to_html(escape=False ,formatters=format_dict)))

refugio.RAZA.replace({'galga': 'galgo', 'mestiza': 'mestizo'}, inplace= True)
refugio.TAMAÑO.replace({'medio': 'mediano', 'mediana': 'mediano'}, inplace= True)
refugio.CARACTER.replace({'muy cariñosa': 'cariñosa', 'timida': 'tímida'}, inplace= True)
refugio['NACIMIENTO'] = pd.to_datetime(refugio['NACIMIENTO'])
refugio['NACIMIENTO'] = (refugio['NACIMIENTO']).dt.year
refugio['HOY'] = pd.datetime.now().date()
refugio['HOY'] = pd.to_datetime(refugio['HOY'])
refugio['HOY'] = (refugio['HOY']).dt.year
refugio['AÑOS'] = (refugio['HOY'] - refugio['NACIMIENTO'])
refugio.drop(['NACIMIENTO', 'HOY', 'EDAD'], axis = 1, inplace = True)


In [6]:
#PROTECTORAS

In [7]:
colnames= ('ID', 'NOMBRE', 'UBICACION', 'CONTACTO')
protectoras = pd.DataFrame(columns=colnames)
elrefugio= {'ID': 1, 'NOMBRE': 'El Refugio', 'UBICACION': 'MADRID', 'CONTACTO': '(+34) 91 730 3680'}
protectoras = protectoras.append(elrefugio, ignore_index = True)

In [8]:
# ACUNR

In [9]:
url = 'https://www.acunr.es/perros/'
driver=webdriver.Chrome(PATH) 
driver.get(url)

links= []

for e in tqdm(range(5)):
    muchos = driver.find_element(By.CLASS_NAME, 'col-lg-12')
    uno = muchos.find_elements(By.TAG_NAME, 'a')
    links += (e.get_attribute('href') for e in uno)
    otra = driver.find_element(By.XPATH, '//*[@id="main"]/ul[1]/li[8]/a')
    otra.click()
'''
ESTO NO LO PONGO DENTRO DEL BUCLE PORQUE ES LA ULTIMA PAG
SI LO PONGO DENTRO DEL BUCLE, INTENTA DAR A CLICK Y ME DA ERROR
PORQUE NO HAY OTRA PAG
'''
muchos = driver.find_element(By.CLASS_NAME, 'col-lg-12')
uno = muchos.find_elements(By.TAG_NAME, 'a')
links += [e.get_attribute('href') for e in uno]


links_acunr= []
for e in range(len(links)):
    if e % 2:
        links_acunr.append(links[e])
    else:
        pass
    
    

imagen=[]
nombres= []
carac= []
protec_id=[]
for e in tqdm(links_acunr):
        protec_id.append(2) 
        driver.get(e)
        time.sleep(0.2)
        nombres.append(driver.find_element(By.TAG_NAME, 'h2').text)
        box = driver.find_element(By.CLASS_NAME, 'col-lg-12')
        img= box.find_element(By.TAG_NAME, 'img')
        imagen.append(img.get_attribute('src'))
        desc = box.find_elements(By.CLASS_NAME, 'card-text')
        for i in desc:
            if len(desc) == 12:
                desc.insert(9, i)
            else:
                pass
            
            i = i.text
            i = i.split(':', 1)
            i.pop(0)
            carac += i
            
            
            
driver.quit()

n = 13
caracter = [carac[i:i + n] for i in range(0, len(carac), n)]

100%|██████████████████████████████████████████████████████████████████████████████████| 52/52 [02:38<00:00,  3.05s/it]


In [11]:
colnames= ('ID', 'EDAD', 'RAZA', 'SEXO', 'ORIGEN', 'CARACTER', 'APTO_CON_GATOS', 'LEHISMANIA', '4DX', 'SALUD', 'SITUACION', 'TAMAÑO', 'OBSERVACIONES')
acunr_ini = pd.DataFrame(caracter, columns=colnames)
acunr_ini['PROTECTORA']=protec_id
acunr_ini['NOMBRE']= nombres
acunr_ini['LINK']= links_acunr

images1 = imagen
acunr_ini['FOTO'] = images1

# convert your links to html tags 
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

pd.set_option('display.max_colwidth', None)

image_cols = ['FOTO']  #<- define which columns will be used to convert to html

# Create the dictionariy to be passed as formatters
format_dict = {}
for e in image_cols:
    format_dict[e] = path_to_image_html


# display(HTML(acunr_ini.to_html(escape=False ,formatters=format_dict)))

acunr = acunr_ini.copy()

años= []
for e in acunr.EDAD:
    e = e.lower()
    if ' adulto' in e or ' senior' in e or ' yayo' in e:
        años.append(int(re.findall('[0-9]+', e)[0]))
    elif ' cachorro' in e:
        años.append(0)
    else:
        años.append(e)
acunr['AÑOS'] = años

raza= []
for e in acunr.RAZA:
    e = e.lower()
    e= e.strip()
    raza.append(e)
acunr['RAZA']= raza

sexo= []
for e in acunr.SEXO:
    e = e.lower()
    e= e.strip()
    sexo.append(e)
acunr['SEXO']= sexo

tam= []
for e in acunr.TAMAÑO:
    e = e.lower()
    e= e.strip()
    tam.append((re.findall('[a-z,ñ]+', e)[0]))
acunr['TAMAÑO']= tam

nom= []
for e in acunr.NOMBRE:
    e = e.capitalize()
    e= e.strip()
    nom.append(e)
acunr['NOMBRE']= nom


acunr.drop(['EDAD', 'ID', 'ORIGEN', 'APTO_CON_GATOS', 'LEHISMANIA', '4DX', 'SALUD', 'SITUACION', 'OBSERVACIONES'], axis= 1, inplace = True)

In [16]:
perros = pd.concat([refugio, acunr])

In [17]:
perros

,SEXO,RAZA,TAMAÑO,CARACTER,NOMBRE,PROTECTORA,LINK,FOTO,AÑOS
0,macho,mestizo,grande,juguetón,Random,1,https://elrefugio.org/ficha/random,https://elrefugio.org/contenido/animales/thumbsFicha/El-Refugio-Randomweb_1.jpg,6
1,hembra,galgo,mediano,cariñosa,Kaysa,1,https://elrefugio.org/ficha/kaysa,https://elrefugio.org/contenido/animales/thumbsFicha/El-Refugio-Kaysaweb.jpg,1
2,hembra,mestizo,mediano,tímida,Tex,1,https://elrefugio.org/ficha/tex,https://elrefugio.org/contenido/animales/thumbsFicha/El-Refugio-Texweb.jpg,5
3,macho,mestizo,mediano,activo,Rex,1,https://elrefugio.org/ficha/rex,https://elrefugio.org/contenido/animales/thumbsFicha/El-Refugio-Rexweb.jpg,2
4,macho,mestizo,mediano,cariñoso,Ommo,1,https://elrefugio.org/ficha/ommo,https://elrefugio.org/contenido/animales/thumbsFicha/El-Refugio-Ommoweb.jpg,0
...,...,...,...,...,...,...,...,...,...
47,macho,otras razas mestizo,grande,"King padece mucho estrés en el albergue que le impide relacionarse bien con la mayoría de los perros, en la medida que podemos trabajamos con el, pero obviamente el entorno no es el adecuado y los avances son pocos. Cuando se relaja es un perro cariñoso, muy inteligente al que le encanta interactuar con la gente, pero necesita un trabajo rodeado de calma y estabilidad que en el albergue no podemos ofrecerle.",King,2,https://www.acunr.es/ficha-perros/?id=27&idGaleria=33,https://www.acunr.es/wp-content/gallery/portadas/27_KING_ACUNR.jpg,7
48,macho,otras razas mestizo rotwailer,mediano,"Koldo llegó a nosotros siendo un cachorro aun, creció abandonado en la calle comiendo en los contenedores. A día de hoy, después de muchos meses ya con nosotros aun no nos deja tocarle, no sabemos qué trauma arrastra después de pasar sus primeros meses abandonado. Pero gracias a unos voluntarios que con constancia, paciencia y cariño trabajan con el, poco a poco va avanzando, ya hemos conseguido que acepte que le pongamos la correa para poder sacarle a pasear, de lo cual disfruta enormemente, también le encanta jugar a la pelota; se pone feliz cuando la ve y te la deja a los pies esperando a que la lances. Con sus compañeros de recinto, que lamentablemente ya han sido muchos desde que está con nosotros, tienen un comportamiento envidiable, sociable y equilibrado. No sabemos exactamente que le pasó a Koldo para no permitir el contacto con nosotros, pero si sabemos que por el contrario, es un perro al que le encanta estar con las personas, aprender e interactuar con nosotros, así que buscamos a esa persona, que quiera descubrir y ayudar a salir a la superficie... a este bombón de perro. >>><<<< INFORMACIÓN IMPORTANTE para la ADOPCIÓN de KOLDO Es muy IMPORTANTE que en su familia haya otro perro, le ayudan mucho a sentirse más confiado y le sirven de referencia. De igual manera es importante que disponga de un jardín o espacio donde él se sienta que puede tomar distancia para estar más seguro. >>><<< Desde ACUNR PRESTAREMOS TODO NUESTRO APOYO, AYUDA Y SEGUIMIENTO PARA SU ADAPTACIÓN, esto incluye un periodo previo a la adopción en el que para crear vínculo con Koldo consideramos muy importante conocerle en el albergue, organizaremos con los adoptantes algunas visitas previas destinadas a que os conozcáis, le paseéis, juguéis con él, etc. Visitas compartidas siempre con uno de los voluntarios que mejor conoce a Koldo, y que será a su vez quien haga el posterior seguimiento y apoyo a la adopción.",Koldo,2,https://www.acunr.es/ficha-perros/?id=20&idGaleria=25,https://www.acunr.es/wp-content/gallery/portadas/koldo.jpg,4
49,macho,otras razas mestizo,mediano,"Hache es un perro sociable, activo y juguetón al que le encanta la gente.",Hache,2,https://www.acunr.es/ficha-perros/?id=13&idGaleria=330,https://www.acunr.es/wp-content/gallery/portadas/286_HACHE_ACUNR_1.jpg,5
50,hembra,otras razas mestiza american staffordshire,mediano,"Kat es un amor de perra. Es cariñosa, alegre, juguetona, divertida y sociable. Le encantan los mimos y salir de paseo. Busca constantemente caricias y te da be